# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [2]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [3]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
    # join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

    # reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
        for line in csvreader:
            full_data_rows_list.append(line) 
            
# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [5]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Design of Apache Cassandra tables. 

The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [6]:
from cassandra.cluster import Cluster
cluster = Cluster(["127.0.0.1"])
session = cluster.connect()

#### Create Keyspace

In [7]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify   
    WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }""")
except Exception as e:
    print(e)

#### Set Keyspace

In [8]:
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

## **Query 1**

##### Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4



- Primary key of the table for this query should consist of one partition and one clustering columns
    - Partition column will be session_id, thus data will be distributed to the nodes based on session_id
    - Clustering columns will be item_in_session and data will be sorted within partition based on it

In [9]:
query = """
        CREATE TABLE IF NOT EXISTS songs 
        (session_id int, item_in_session int, artist text, song_title text, duration float,
        PRIMARY KEY (session_id, item_in_session))
        """
try:
    session.execute(query)
except Exception as e:
    print(e)

In [58]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO songs (session_id, item_in_session, artist, song_title, duration)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Verification select for Query 1

In [59]:
try:
    rows = session.execute("SELECT artist, song_title, duration FROM songs WHERE session_id=338 AND item_in_session=4")
except Exception as e:
    print(e)
    
result_df = pd.DataFrame(list(rows))
result_df

,artist,song_title,duration
0,Faithless,Music Matters (Mark Knight Dub),495.307312


## Query 2

##### Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

- Primary key of the table for this query should consist of two partition and one clustering columns
    - Partition columns will be user_id and session_id, thus data will be distributed to the nodes based on this two keys
    - Clustering column will be item_in_session because we want to sort our songs based on this columns

In [11]:
query = """
        CREATE TABLE IF NOT EXISTS usage_log
        (user_id int, session_id int,  item_in_session int, artist text, song_title text, user_name text, user_surname text, 
        PRIMARY KEY ((user_id, session_id), item_in_session))
        """
try:
    session.execute(query)
except Exception as e:
    print(e)

In [12]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO usage_log (user_id, session_id, item_in_session, artist, song_title, user_name, user_surname)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

#### Verification select for Query 2

In [13]:
try:
    rows = session.execute("SELECT artist, song_title, user_name, user_surname FROM usage_log WHERE user_id=10 AND session_id=182")
except Exception as e:
    print(e)
    
result_df = pd.DataFrame(list(rows))
result_df

,artist,song_title,user_name,user_surname
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


### Query 3

##### Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'¶

- Primary key of the table for this query should consist of two partition columns
    - Partition columns will be song_title and user_id, this will make every database entry unique and allow to execute desired query

In [18]:
query = """
        CREATE TABLE IF NOT EXISTS users
        (song_title text, user_name text, user_surname text, user_id int, 
        PRIMARY KEY (song_title, user_id))
        """
try:
    session.execute(query)
except Exception as e:
    print(e)

In [20]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO users (song_title, user_name, user_surname, user_id)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], line[1], line[4], int(line[10])))

In [21]:
try:
    rows = session.execute("SELECT song_title, user_name, user_surname FROM users WHERE song_title='All Hands Against His Own'")
except Exception as e:
    print(e)
    
result_df = pd.DataFrame(list(rows))
result_df

,song_title,user_name,user_surname
0,All Hands Against His Own,Jacqueline,Lynch
1,All Hands Against His Own,Tegan,Levine
2,All Hands Against His Own,Sara,Johnson


### Drop the tables before closing out the sessions

In [22]:
tables = ["users", "songs", "usage_log"]

for table_name in tables:
    query = f"drop table {table_name}"
    try:
        rows = session.execute(query)
    except Exception as e:
        print(e)

### Close the session and cluster connection¶

In [23]:
session.shutdown()
cluster.shutdown()